In [5]:
require 'hdf5'

{
  H5Z_FILTER_CONFIG_ENCODE_ENABLED : 1
  H5F_ACC_RDWR : 1
  _getTorchType : function: 0x04005888
  H5F_OBJ_FILE : 1
  H5S_ALL : 0
  H5F_OBJ_GROUP : 4
  C : userdata: 0x047f8618
  H5P_DEFAULT : 0
  _describeObject : function: 0x047f3638
  H5Z_FILTER_NBIT : 5
  _debugMode : false
  _getObjectType : function: 0x047ee930
  H5F_OBJ_ALL : 31
  _getObjectName : function: 0x03f58100
  version : 
    {
      1 : 1
      2 : 8
      3 : 15
    }
  H5Z_FILTER_SHUFFLE : 2
  HDF5Group : table: 0x04461da0
  open : function: 0x04471cc8
  H5Z_FILTER_SZIP : 4
  H5F_OBJ_ATTR : 16
  H5Z_FILTER_FLETCHER32 : 3
  H5F_OBJ_DATATYPE : 8
  debugMode : function: 0x04471c48
  H5F_ACC_EXCL : 4
  H5Z_FILTER_NONE : 0
  _testUtils : 
    {
      deepAlmostEq : function: 0x04471c08
      withTmpDir : function: 0x04471b68
    }
  _nativeTypeForTensorType : function: 0x03f580b8
  H5F_ACC_TRUNC : 2
  _config : 
    {
      HDF5_INCLUDE_PATH : /Users/nicolasdrizard/anaconda/include
      HDF5_LIBRARIES : /Users/nicolasd

      NATIVE_HSSIZE : 50331699
      STD_B32BE : 50331729
      INTEGER : 0
      NATIVE_INT64 : 50331682
      STD_I8LE : 50331708
      STD_I32LE : 50331712
      NATIVE_SCHAR : 50331656
      NATIVE_INT_FAST16 : 50331674
      NATIVE_INT : 50331660
      BITFIELD : 4
      NATIVE_UINT_LEAST16 : 50331673
      NATIVE_INT_LEAST16 : 50331672
      IEEE_F64LE : 50331704
      NATIVE_INT_FAST64 : 50331686
      NATIVE_UINT_FAST64 : 50331687
      NATIVE_UINT_LEAST64 : 50331685
      NATIVE_INT_LEAST64 : 50331684
      ENUM : 8
      NATIVE_UINT64 : 50331683
      NATIVE_HBOOL : 50331701
      NATIVE_ULONG : 50331663
      NATIVE_INT_FAST32 : 50331680
      NATIVE_HADDR : 50331697
      NATIVE_UINT : 50331661
      NCLASSES : 11
      NATIVE_UINT_LEAST32 : 50331679
      NATIVE_INT_LEAST32 : 50331678
      STD_U64LE : 50331722
      NATIVE_UINT32 : 50331677
      NATIVE_SHORT : 50331658
      NATIVE_INT32 : 50331676
      VLEN : 9
      ARRAY : 10
      STD_U16LE : 50331718
      STD_B16L

In [24]:
myFile = hdf5.open('../data_preprocessed/5-grams.hdf5','r')
data = myFile:all()

In [25]:
input_matrix_train = data['input_matrix_train']
output_matrix_train = data['output_matrix_train']
F_train = data['F_train']
input_data_valid = data['input_data_valid']
input_data_test = data['input_data_test']
myFile:close()

In [26]:
F_train[1]

  4
 22
  3
  4
 31
 58
[torch.DoubleTensor of size 6]



In [41]:
-- Build the mapping from (N-1)gram to row index
N = 5
ngram_to_ind = {}
for i=1,F_train:size(1) do
    ngram_to_ind[F_train[i]:narrow(1,1,N-1)] = i
end

In [ ]:
input_data_valid

In [40]:

for 1=1,input_data_test

1	


In [ ]:
function predict_count_based(gram_input, F)
    